In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K
import sys
import datetime
from tensorflow.python.client import device_lib

from sificc_lib import AI, Utils_new, Event_new, Simulation_new, root_files, DataModel_new
np.set_printoptions(precision=2, linewidth=85, suppress=True)

%matplotlib inline

In [2]:
# Check used versions
now = datetime.datetime.now()
print("Date-time           ", now)
print("Tf version:         ", keras.__version__)
print("Numpy version:      ", np.__version__)
print("Python              ", sys.executable)
print("Python version      ", sys.version)
print("GPU devices:        ", device_lib.list_local_devices())

Date-time            2021-07-30 10:50:34.599805
Tf version:          2.3.0-tf
Numpy version:       1.19.1
Python               /net/software_g4rt/anaconda3/envs/condaenv-sificcnn/bin/python
Python version       3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]
GPU devices:         [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16395677361470279912
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1237031293368460326
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
# model name
model_name = 'model-2b-lsr-new-real'

shuffle_clusters = False

# load the training data
data = DataModel_new('data-new-complete-real.npz', 
                 batch_size = 128, validation_percent = .05, test_percent = .1)

# append an extra dimention to the features since we are using convolutional layers
data.append_dim = True

# create an AI instance
ai = AI(data, model_name)

ai.weight_type = 2
ai.weight_pos_x = 2.5
ai.weight_pos_y = 1
ai.weight_pos_z = 2
ai.weight_energy = 1.5
ai.weight_e_cluster = 1
ai.weight_p_cluster = 1

# randomly shuffle the training data
#np.random.seed(888)
#ai.data.shuffle(only_train=False)

# shuffle the clusters within each event
if shuffle_clusters:
    ai.data.shuffle_training_clusters()

In [4]:
# define and create the neural network architecture
ai.create_model(conv_layers=[128, 64], classifier_layers=[32], type_layers=[8], 
                pos_layers=[64,32], energy_layers=[32, 16], base_l2=.0001, limbs_l2=.0001)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 72, 1)]      0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 8, 128)       1280        inputs[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 8, 64)        8256        conv_1[0][0]                     
__________________________________________________________________________________________________
flatting (Flatten)              (None, 512)          0           conv_2[0][0]                     
______________________________________________________________________________________________

In [5]:
# compile the ai
ai.compile_model(learning_rate=0.0003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 90:
        return .0003
    elif epoch < 160:
        return .0001
    elif epoch < 180:
        return .00003
    elif epoch < 190:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

In [6]:
#LOADING after training
#ai.load(model_name, optimizer=False)
#ai.compile_model()

In [7]:
#TRAINING

In [8]:
%%time
# start the training
ai.train(epochs=200, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)

# evaluate the AI on the training set
ai.model.evaluate(ai.data.train_x, ai.data.train_y, verbose=1)
print()

9737/9737 [==============================] - 47s 5ms/step - loss: 7.5593 - e_cluster_loss: 0.0445 - p_cluster_loss: 0.1230 - type_loss: 0.3691 - pos_x_loss: 0.6923 - pos_y_loss: 2.4523 - pos_z_loss: 0.2904 - energy_loss: 1.2309 - e_cluster__cluster_accuracy: 0.9547 - p_cluster__cluster_accuracy: 0.8236 - type__type_accuracy: 0.8321 - type__type_tp_rate: 0.5315

CPU times: user 11min 44s, sys: 28min 16s, total: 40min
Wall time: 5min 16s


In [9]:
# plot the training loss
#ai.plot_training_loss(smooth=False)

In [10]:
#ai.plot_training_loss(mode = 'loss')

In [11]:
# evaluate the AI on the test dataset
#ai.evaluate()

In [12]:
#ai.savefigpath = '/home/home2/institut_3b/profe/Data/SiFiCCNN/21-06_EventAnalysis_ClusterNumbers_EnergyDepo/model-2b-lsr/EnergyDeposition/'
#ai.events_prediction_analysis('all-events', save = True)

In [13]:
#ai.events_prediction_analysis('cluster-numbers')

In [14]:
#ai.events_prediction_analysis('cluster-distribution')

In [15]:
#ai.events_prediction_analysis('e-scatterer')

In [17]:
# save the trained model
ai.save(file_name = model_name)

In [18]:
#ai.load(model_name, optimizer=False)